In [43]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Based on https://www.kaggle.com/code/manikg/training-svm-classifier-with-hog-features
for the dataset https://www.kaggle.com/datasets/rhammell/ships-in-satellite-imagery

In [44]:
import json
with open('../input/ships-in-satellite-imagery/shipsnet.json') as data_file:
    dataset = json.load(data_file)
ships = pd.DataFrame(dataset)

In [45]:
ships.head()

In [46]:
from skimage import color

def make_gray(data):
    img_length = 80
    array = np.array(data).reshape(-1,3,img_length,img_length).transpose([0,2,3,1])
    return color.rgb2gray(np.squeeze(array))

In [47]:
ships['gray_data'] = ships['data'].transform(make_gray)

In [48]:
from skimage.feature import hog

def get_hog(data):
    ppc = 16
    features,hog_image = hog(np.array(data), orientations=8, pixels_per_cell=(ppc,ppc),cells_per_block=(4, 4),block_norm= 'L2',visualize=True)
    return np.array(features)

In [49]:
ships['features'] = ships['gray_data'].transform(get_hog)

In [50]:
from sklearn import svm
from sklearn.model_selection import train_test_split
ships = ships.sample(frac=1).reset_index(drop=True)

In [72]:
X_train, X_test, y_train, y_test = train_test_split(ships['features'], ships['labels'], test_size=0.2, random_state=1)

In [74]:
model = svm.SVC()
model.fit(list(X_train),y_train)

In [75]:
y_pred = model.predict(list(X_test))

In [76]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [79]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))